In [ ]:
using PMP, DataFrames, Distributions, Gadfly, Cairo, Fontconfig, Statistics

This notebook creates multiple samples out of different Pearson type I distributions with a=0 fixed then fits models to those samples. We use 4 distributions, of parameters
- A: b = 50, $\alpha$ = 2, $\beta$ = 2;
- B: b = 50, $\alpha$ = 1, $\beta$ = 3;
- C: b = 50, $\alpha$ = 0.1, $\beta$ = 6;
- D: b = 50, $\alpha$ = 0.1, $\beta$ = 15
For each of these distributions and for 6 different sizes (100, 500, 1000, 2000, 5000, 8000), we create 100 samples.

This code is not optimized. Particularly for the distributions C and D, the running time is very long.


---
# 3 parameters Pearson type I
---

# Distribution A

## Parameters b = 50, $\alpha$ = 2, $\beta$ = 2
Creating samples

In [ ]:
d1 = PearsonType1b(50, 2, 2)
taille = [100, 500, 1000, 2000, 5000, 8000];

In [ ]:
nb = 100
sample_a = Array{Float64, 2}(undef, 100, taille[1])
sample_b = Array{Float64, 2}(undef, 100, taille[2])
sample_c = Array{Float64, 2}(undef, 100, taille[3])
sample_d = Array{Float64, 2}(undef, 100, taille[4])
sample_e = Array{Float64, 2}(undef, 100, taille[5])
sample_f = Array{Float64, 2}(undef, 100, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d1, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
end

df1 = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end

df1.bq2_5 = quantile_b[1,:]
df1.bq97_5 = quantile_b[2,:]

df1.αq2_5 = quantile_α[1,:]
df1.αq97_5 = quantile_α[2,:]

df1.βq2_5 = quantile_β[1,:]
df1.βq97_5 = quantile_β[2,:]

df1

In [ ]:
p1 = plot(df1, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
        Coord.cartesian(ymin=40, ymax=75),
    Guide.Title("MM"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p1

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1b, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df1m = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1m, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df1m.bq2_5 = quantile_b[1,:]
df1m.bq97_5 = quantile_b[2,:]

df1m.αq2_5 = quantile_α[1,:]
df1m.αq97_5 = quantile_α[2,:]

df1m.βq2_5 = quantile_β[1,:]
df1m.βq97_5 = quantile_β[2,:]

df1m

In [ ]:
p1m = plot(df1m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
            Coord.cartesian(ymin=40, ymax=75),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df1mle = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df1mle, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df1mle.bq2_5 = quantile_b[1,:]
df1mle.bq97_5 = quantile_b[2,:]

df1mle.αq2_5 = quantile_α[1,:]
df1mle.αq97_5 = quantile_α[2,:]

df1mle.βq2_5 = quantile_β[1,:]
df1mle.βq97_5 = quantile_β[2,:]

df1mle

In [ ]:
p1mle = plot(df1mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
            Coord.cartesian(ymin=40, ymax=75),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))  

In [ ]:
t1 = title(hstack(p1, p1m, p1mle), "Distribution A")


---
# Distribution B

## Parameters : b = 50, $\alpha$ = 1, $\beta$ = 3
Creating samples

In [ ]:
d2 = PearsonType1b(50, 1, 3)

sample_a = Array{Float64, 2}(undef, 100, taille[1])
sample_b = Array{Float64, 2}(undef, 100, taille[2])
sample_c = Array{Float64, 2}(undef, 100, taille[3])
sample_d = Array{Float64, 2}(undef, 100, taille[4])
sample_e = Array{Float64, 2}(undef, 100, taille[5])
sample_f = Array{Float64, 2}(undef, 100, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d2, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
end

df2 = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df2.bq2_5 = quantile_b[1,:]
df2.bq97_5 = quantile_b[2,:]

df2.αq2_5 = quantile_α[1,:]
df2.αq97_5 = quantile_α[2,:]

df2.βq2_5 = quantile_β[1,:]
df2.βq97_5 = quantile_β[2,:]

df2

In [ ]:
p2 = plot(df2, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Coord.cartesian(ymin=30, ymax=120),
    Guide.Title("MM"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p2

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1b, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df2m = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2m, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df2m.bq2_5 = quantile_b[1,:]
df2m.bq97_5 = quantile_b[2,:]

df2m.αq2_5 = quantile_α[1,:]
df2m.αq97_5 = quantile_α[2,:]

df2m.βq2_5 = quantile_β[1,:]
df2m.βq97_5 = quantile_β[2,:]

df2m

In [ ]:
p2m = plot(df2m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Coord.cartesian(ymin=30, ymax=120),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p2m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df2mle = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df2mle, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df2mle.bq2_5 = quantile_b[1,:]
df2mle.bq97_5 = quantile_b[2,:]

df2mle.αq2_5 = quantile_α[1,:]
df2mle.αq97_5 = quantile_α[2,:]

df2mle.βq2_5 = quantile_β[1,:]
df2mle.βq97_5 = quantile_β[2,:]

df2mle

In [ ]:
p2mle = plot(df2mle, x=:taille, y=:b, Geom.point,
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Coord.cartesian(ymin=30, ymax=120),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p2mle

In [ ]:
t2 = title(hstack(p2, p2m, p2mle), "Distribution B")


---
# Distribution C
## Parameters b = 50, $\alpha$ = 0.1, $\beta$ = 6

Creating samples

In [ ]:
d3 = PearsonType1b(50, 0.1, 6)

nb = 100
sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d3, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
end

df3 = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df3.bq2_5 = quantile_b[1,:]
df3.bq97_5 = quantile_b[2,:]

df3.αq2_5 = quantile_α[1,:]
df3.αq97_5 = quantile_α[2,:]

df3.βq2_5 = quantile_β[1,:]
df3.βq97_5 = quantile_β[2,:]

df3

In [ ]:
p3 = plot(df3, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
        Coord.cartesian(ymin=0, ymax=200),
    Guide.Title("MM"), Guide.xlabel("Size"), Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1b, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df3m = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3m, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df3m.bq2_5 = quantile_b[1,:]
df3m.bq97_5 = quantile_b[2,:]

df3m.αq2_5 = quantile_α[1,:]
df3m.αq97_5 = quantile_α[2,:]

df3m.βq2_5 = quantile_β[1,:]
df3m.βq97_5 = quantile_β[2,:]

df3m

In [ ]:
p3m = plot(df3m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
            Coord.cartesian(ymin=0, ymax=200),
    Guide.Title("Mixed"), Guide.xlabel("Size"), Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df3mle = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df3mle, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df3mle.bq2_5 = quantile_b[1,:]
df3mle.bq97_5 = quantile_b[2,:]

df3mle.αq2_5 = quantile_α[1,:]
df3mle.αq97_5 = quantile_α[2,:]

df3mle.βq2_5 = quantile_β[1,:]
df3mle.βq97_5 = quantile_β[2,:]

df3mle

In [ ]:
p3mle = plot(df3mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p3mle

In [ ]:
t3 = title(hstack(p3, p3m, p3mle), "Distribution C")


---
# Distribution D
## Parameters : b = 50, $\alpha$ = 0.1, $\beta$ = 15
Creating samples

In [ ]:
d4 = PearsonType1b(50, 0.1, 15)

nb = 100
sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d4, taille[j])
    end
end

### Moment method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]


for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mme(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
end

df4 = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df4.bq2_5 = quantile_b[1,:]
df4.bq97_5 = quantile_b[2,:]

df4.αq2_5 = quantile_α[1,:]
df4.αq97_5 = quantile_α[2,:]

df4.βq2_5 = quantile_β[1,:]
df4.βq97_5 = quantile_β[2,:]

df4

In [ ]:
p4 = plot(df4, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
            Coord.cartesian(ymin=0, ymax=700),
    Guide.Title("MM"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p4

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1b, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df4m = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4m, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df4m.bq2_5 = quantile_b[1,:]
df4m.bq97_5 = quantile_b[2,:]

df4m.αq2_5 = quantile_α[1,:]
df4m.αq97_5 = quantile_α[2,:]

df4m.βq2_5 = quantile_β[1,:]
df4m.βq97_5 = quantile_β[2,:]

df4m

In [ ]:
p4m = plot(df4m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
                Coord.cartesian(ymin=0, ymax=700),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p4m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_b = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_c = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_d = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_e = DataFrame(b = Float64[], α = Float64[], β = Float64[])
params_f = DataFrame(b = Float64[], α = Float64[], β = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1b, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], α=fit[2], β=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df4mle = DataFrame(taille = taille[1], b = mean(params_a.b), α = mean(params_a.α), β = mean(params_a.β))

for j in 2:length(taille)
    append!(df4mle, DataFrame(taille=taille[j], b=mean(names[j].b),  α=mean(names[j].α), β=mean(names[j].β)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_α = Array{Float64, 2}(undef, 2, 6)
quantile_β = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_α[:,j] = quantile(names[j].α, [.025, .975])
    quantile_β[:,j] = quantile(names[j].β, [.025, .975])
end


df4mle.bq2_5 = quantile_b[1,:]
df4mle.bq97_5 = quantile_b[2,:]

df4mle.αq2_5 = quantile_α[1,:]
df4mle.αq97_5 = quantile_α[2,:]

df4mle.βq2_5 = quantile_β[1,:]
df4mle.βq97_5 = quantile_β[2,:]

df4mle

In [ ]:
p4mle = plot(df4mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt)) 
p4mle

In [ ]:
t4 = title(hstack(p4, p4m, p4mle), "Distribution D")
t4